## Get Data

To download all match data from static dataset. Run `python lol/download_static_data.py` this will store all static data into `lol/static_matches.data`.

#### Load saved data

In [1]:
import utils

In [2]:
static_matches = utils.load_data('lol/static_matches.data')

## Build a network

In [3]:
import networkx as nx
import time
from collections import deque

In [4]:
import algos
reload(algos)

<module 'algos' from 'algos.pyc'>

In [5]:
from lol import riot_games_api
reload(riot_games_api)

<module 'lol.riot_games_api' from 'lol/riot_games_api.pyc'>

In [6]:
rg = riot_games_api.RiotGames('lol/riot_games_api.key')

#### Find all team mates for one summoner

In [7]:
# Seed node
summonerId = static_matches[0]['participantIdentities'][0]['player']['summonerId']

In [8]:
# Initialize -- Starting from seed
matches = {}  # Store data for all matches
discovered_summonerIds = {summonerId: True}  # Track discovered nodes
g = nx.Graph()

max_hop = 2  # Used as a stopping condition for Breadth First Search

bfs_queue = deque([{'summonerId': summonerId, 'hop': 0}])  # Queue for breadth first search
hop = 0

In [30]:
# Optional: Initialize from saved state

matches = utils.load_data('matches')
discovered_summonerIds = utils.load_data('discovered_summonerIds')
g = utils.load_data('g')
max_hop = utils.load_data('max_hop')
bfs_queue = utils.load_data('bfs_queue')
hop = utils.load_data('hop')

In [ ]:
while (len(bfs_queue)>0) and (hop <= max_hop):
    # Dequeue next summonerId
    next_item_in_queue = bfs_queue.popleft()
    summonerId = next_item_in_queue['summonerId']
    hop = next_item_in_queue['hop']
    
    # Get match ids for current node
    matchIds = rg.get_matchIds_by_summoner(summonerId)
    num_matches = len(matchIds)
    
    print ""
    print "SummonerId %d at hop %d has %d matches." % (summonerId, hop, num_matches)
    
    # Loop through all matches and add all members to the network
    for i, matchId in enumerate(matchIds):
        print "\rMatch %d out of %d [%0.1f%%]" % (i+1, num_matches, (i+1)/float(num_matches)*100),
        
        if matchId in matches:  # skip loop if match info already retreived
            continue

        # Get full match data and extract team members
        try:
            match = rg.get_match(matchId)
        except:
            print "Error, skipping matchId: %d" % matchId
            
        team_memberIds = rg.get_summonerIds_by_match(match=match, team_with=summonerId)

        # Add team members to graph as a clique, with matchId info on edges
        g = algos.add_clique_with_weights(g, team_memberIds, edge_attr={'matchId': matchId}) 

        # Add new memberIds to queue
        for i in team_memberIds:
            if i not in discovered_summonerIds:
                bfs_queue.append({'summonerId': i, 'hop': hop+1})
                discovered_summonerIds[i] = True

        # Add match data to matches dict
        match = utils.list_of_dict_to_dict([match], 'matchId')
        matches.update(match)
        
        # Sleep to stay under the API data rate limit
        time.sleep(1.2)        


SummonerId 21284763 at hop 1 has 479 matches.
Match 73 out of 479 [15.2%]

In [25]:
# Save state
utils.save_data(matches, 'matches')
utils.save_data(discovered_summonerIds, 'discovered_summonerIds')
utils.save_data(g, 'g')
utils.save_data(max_hop, 'max_hop')
utils.save_data(bfs_queue, 'bfs_queue')
utils.save_data(hop, 'hop')

In [22]:
len(matches)

5044

In [23]:
len(bfs_queue)

17282

In [24]:
print "Network as %d nodes and %d edges" % (len(g.nodes()), len(g.edges()))

Network as 17288 nodes and 46392 edges
